In [76]:
# add path to module so can import mlfinder code
import sys
sys.path = ['/Users/judahluberto/mlfinder'] + sys.path

# ignore future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# basic imports
from astropy.io import ascii
from astropy.io import fits
from astropy.table import Table
from astropy.time import Time
from astropy.coordinates import SkyCoord
import astropy.units as u

import math

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

import statistics

from PyAstronomy import pyasl

import os

# datalab imports for background stars
import dl
from dl import queryClient as qc
from dl.helpers.utils import convert

# import mlfinder module
from mlfinder.bd import BrownDwarf
from mlfinder.fields import Fields
from mlfinder.events import FindEvents

In [162]:
# get dataframe

df = Table.read('J_AcA_68_351_events.dat.fits', format='fits')
df = df.to_pandas()

df['Disc'] = df['Disc'].str.decode("utf-8")

# make a copy of df, so can reference later
df_all = df.copy()

# rename and select columns in dataframe
rename_dict = {'Disc':'object_name', 
               'RALdeg':'ra', 
               'DELdeg':'dec',
               'EpL': 'EpL',
               't0': 'Prev_Pred_Min_Sep',
               'PlxL':'pi', 
               'pmRA*L':'mu_alpha', 
               'pmDEL':'mu_delta', 
               'e_RALdeg':'pm_ra',
               'e_DELdeg':'pm_dec',
               'e_pmRA*L':'pm_mu_alpha',
               'e_pmDEL':'pm_mu_delta',
               'e_PlxL': 'pm_pi'}

df = df.rename(columns = rename_dict)

# now keep those columns too -- for simplicity!
keep_list = ['object_name', 'ra', 'dec', 'EpL', 'pi', 'mu_alpha', 'mu_delta', 'pm_mu_alpha', 'pm_mu_delta', 'pm_pi', 'Prev_Pred_Min_Sep', 'JmagL', 'e_JmagL', 'Gmag', 'e_Gmag']

df = df[keep_list]

# change name of EpL (already had a t0, so have to convoluded way it)
df = df.rename(columns = {'EpL': 't0'})

# add ra and dec error
df['pm_ra'] = 0
df['pm_dec'] = 0

# add IDs to them for ease of recall later (multiple events of same object makes it harder)
df['id'] = range(0, len(df))

In [163]:
df

,object_name,ra,dec,t0,pi,mu_alpha,mu_delta,pm_mu_alpha,pm_mu_delta,pm_pi,Prev_Pred_Min_Sep,JmagL,e_JmagL,Gmag,e_Gmag,pm_ra,pm_dec,id
0,PSO_J076.7092+52.6087,76.709193,52.608725,2010.87847,59.28,57.0,-207.2,4.6,3.0,5.27,2044.71923,15.75,0.07,18.6940,0.0028,0,0,0
1,WISEA_J053257.29+041842.5,83.238910,4.311572,2012.36983,31.63,276.2,-440.6,1.7,2.0,3.81,2052.51581,15.44,0.06,17.4666,0.0014,0,0,1
2,2MASS_J05591914-1404488,89.831895,-14.081459,2011.90516,96.60,569.0,-339.6,2.8,2.0,1.00,2056.08378,13.80,0.02,20.0007,0.0081,0,0,2
3,WISE_J070159.79+632129.2,105.499205,63.358112,2010.47422,60.53,-17.3,-261.7,3.1,4.4,6.19,2062.25955,15.79,0.07,20.5713,0.0094,0,0,3
4,DENIS-P_J0751164-253043,117.815211,-25.511497,2011.44597,56.30,-880.7,150.1,1.2,1.5,0.09,2045.78042,13.16,0.02,20.9383,0.0162,0,0,4
5,VVV_BD001,261.667255,-27.634409,2012.02303,57.00,-548.7,-329.6,2.2,4.4,4.00,2036.93087,13.40,0.03,20.8126,0.0190,0,0,5
6,2MASS_J18284076+1229207,277.169077,12.488828,2011.31003,16.36,-244.4,-90.2,2.3,1.4,2.72,2061.19042,14.61,0.04,17.7971,0.0013,0,0,6
7,vB_10,289.238239,5.146342,2010.72041,168.15,-594.9,-1364.1,2.2,1.9,0.50,2027.06995,9.91,0.03,20.2936,0.0105,0,0,7
8,vB_10,289.238239,5.146342,2010.72041,168.15,-594.9,-1364.1,2.2,1.9,0.50,2065.73963,9.91,0.03,20.0838,0.0074,0,0,8
9,vB_10,289.238239,5.146342,2010.72041,168.15,-594.9,-1364.1,2.2,1.9,0.50,2055.34675,9.91,0.03,19.6399,0.0054,0,0,9


In [164]:
# make a df of the stars ra and dec
df_stars = df_all[['RAdeg', 'DEdeg', 'RPmag']]

df_stars = df_stars.rename(columns = {'RAdeg': 'ra', 'DEdeg': 'dec', 'RPmag' : 'dered_mag_g'})

# add necessary rows
df_stars['type'] = 'PSF'
df_stars['ls_id'] = 0

# put to csv
df_stars.to_csv('df_stars.csv')

In [165]:
month_dict = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}

def time_to_good(date, init_format='mjd'):
    # create Time instantiation to turn from MJD to iso
    t = Time(float(date), format=init_format)
    t.format = 'iso'
    t = t.value
    
    # split into month, day, year
    t_split = t.split()[0]
    t_split = t_split.split('-')

    # final reformat
    t = t_split[0] + '-' + str(month_dict[int(t_split[1])]) + '-' + t_split[2]
    
    return t

In [166]:
# initliaze a dataframe to append events to, and mc dict of mass uncertainties if done
all_events = pd.DataFrame()
stars_count = 0
for index, row in df.iterrows():
    # print to keep us updated
    print('Working on dwarf {}, index {}...'.format(row.object_name, index))

    # turn row into a pandas dataframe for ease
    row = row.to_frame().T

    # get time in good format
    t = time_to_good(row.t0, init_format='decimalyear')
    
    t_start = time_to_good(row.Prev_Pred_Min_Sep - 5, init_format='decimalyear')
    t_end = time_to_good(row.Prev_Pred_Min_Sep + 5, init_format='decimalyear')
    
    # create brown dwarf class and find a future path
    bd = BrownDwarf(row, observ_date=t)
    bd_path = bd.find_path(start=t_start, end=t_end, step='7days')

    # find the background stars from the Legacy Survey
    stars = Fields(bd=bd, file='df_stars.csv')
    
    if len(stars.stars) > 0:
        stars_count += 1
    
    # find events with mass uncertainty lower than 1000 Mjup
    events = FindEvents(bd, stars, 1000)
    
    # add id to event table
    table = events.event_table
    table['id'] = row.id.item()

    # append to dataframe
    all_events = all_events.append(table)

Working on dwarf PSO_J076.7092+52.6087    , index 0...


/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


Working on dwarf WISEA_J053257.29+041842.5, index 1...


/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


Working on dwarf 2MASS_J05591914-1404488  , index 2...


...

/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


Working on dwarf vB_10                    , index 25...


/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


Working on dwarf GJ_1245B                 , index 26...


/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


In [167]:
# reformat all_events
event_objects, event_sep, event_deltam = list(), list(), list()
for index, row in all_events.iterrows():
    # grab item 
    event_objects.append(row.object_name.item())
    
    # sep and delta_m
    if type(row.sep) is np.ndarray:
        event_sep.append(row.sep[0])
        event_deltam.append(row.delta_m[0])
    else:
        event_sep.append(row.sep)
        event_deltam.append(row.delta_m) 
        
all_events['object_name'] = event_objects
all_events['sep'] = event_sep
all_events['delta_m'] = event_deltam

In [168]:
all_events

,object_name,sep,delta_m,bd_ra,bd_dec,ls_id,bs_ra,bs_dec,mag,time_of_min,gaia_id,id
0,PSO_J076.7092+52.6087,0.015306,6.641031,76.7101,52.606784,0,76.710107,52.606783,0.0,2044.699521,NaN,0
0,WISEA_J053257.29+041842.5,0.074872,60.883302,83.242003,4.306662,0,83.242016,4.306677,0.0,2052.57358,NaN,1
0,2MASS_J05591914-1404488,0.056454,15.031429,89.839086,-14.085634,0,89.839083,-14.08565,0.0,2056.275154,NaN,2
0,WISE_J070159.79+632129.2,0.180893,76.865446,105.498628,63.354414,0,105.498521,63.354398,0.0,2061.340178,NaN,3
0,DENIS-P_J0751164-253043,0.378995,173.143052,117.805949,-25.510068,0,117.805949,-25.510173,0.0,2045.761807,NaN,4
0,VVV_BD001,0.203413,91.787557,261.662975,-27.636686,0,261.66292,-27.636658,0.0,2037.04449,NaN,5
0,vB_10,0.123033,18.819284,289.235558,5.140161,0,289.235524,5.140154,0.0,2027.10883,NaN,7
0,vB_10,0.161219,24.660297,289.230901,5.12948,0,289.230937,5.129453,0.0,2055.308693,NaN,9
0,WISE_J192841.35+235604.9,0.060497,10.611909,292.169789,23.936896,0,292.169786,23.936913,0.0,2043.686516,NaN,10
0,WISE_J200050.19+362950.1,0.309063,56.598551,300.20917,36.499779,0,300.209066,36.499759,0.0,2034.821355,NaN,11


In [169]:
# make subset of dataframe with event object_names
event_rows = pd.DataFrame(df.loc[df['object_name'].isin(event_objects)])

mc_results = dict()
# iterate through all_events
for index, row in all_events.iterrows():
    print('Working on {}'.format(row.object_name))
    
    # find row of object in df
    obj_id = row.id
    obj_row = df[df.id == obj_id]

    # convert row to pd dataframe
    event_row = pd.DataFrame(row).T

    # brown dwarf observe date to create BrownDwarf instance
    t = time_to_good(obj_row.t0.item(), 'decimalyear')

    # cater start and end of brown dwarf path to the event time (+- 0.1 years)
    t_min = round(float(event_row.time_of_min.item()), 2)

    t_start, t_end = time_to_good(t_min - 1, 'decimalyear'), time_to_good(t_min + 1, 'decimalyear')

    # instantiate BrownDwarf
    bd = BrownDwarf(obj_row, observ_date=t)
    bd_path = bd.find_path(start=t_start, end=t_end, step='6h')
    
    # find the background stars from the Legacy Survey
    stars = Fields(bd=bd, file='df_stars.csv')

    # find events with "new" object
    events = FindEvents(bd, stars, 1000)

    # perform the MC with 1000 samples and varying everything
    if len(events.event_table) > 0:
        mass_unc_list, sep_list, time_list, index_list = events.event_mcmc(samples=500, vary=['bd_ra', 'bd_dec', 'pi', 'mu_alpha', 'mu_delta'], prints=100)

        # add mass unc to dict
        mc_results[row.id] = mass_unc_list, sep_list, time_list, index_list

Working on PSO_J076.7092+52.6087    


/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


Working on sample #0...
Working on sample #100...
Working on sample #200...
Working on sample #300...
Working on sample #400...
Working on WISEA_J053257.29+041842.5


/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


Working on sample #0...
Working on sample #100...
Working on sample #200...
Working on sample #300...
Working on sample #400...
Working on 2MASS_J05591914-1404488  


...

Working on sample #0...
Working on sample #100...
Working on sample #200...
Working on sample #300...
Working on sample #400...
Working on vB_10                    
Working on sample #0...
Working on sample #100...
Working on sample #200...
Working on sample #300...
Working on sample #400...
Working on vB_10                    


/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


Working on sample #0...
Working on sample #100...
Working on sample #200...
Working on sample #300...
Working on sample #400...
Working on GJ_1245B                 


/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


Working on sample #0...
Working on sample #100...
Working on sample #200...
Working on sample #300...
Working on sample #400...


In [180]:
for i in mc_results:
    # make a list
    mc_results[i] = list(mc_results[i])
    
    # reassign
    mc_results[i][1] = np.array(mc_results[i][1]) * 1000

In [182]:
for i in mc_results:
    print('Saving {}'.format(i))
    
    # get object_name
    obj_name = df[df.id == i].object_name.item()
    
    # get data
    deltam = mc_results[i][0]
    sep = mc_results[i][1]
    time = mc_results[i][2]
    index = mc_results[i][3]
    
    # file name and path
    filename = os.path.join('/Users/judahluberto/nielson_check', str(i) + '_' + obj_name)
    
    np.savez(filename, deltam=deltam, sep=sep, time=time, index=index)

Saving 0
Saving 1
Saving 2
Saving 3
Saving 4
Saving 5
Saving 7
Saving 9
Saving 10
Saving 11
Saving 12
Saving 13
Saving 14
Saving 15
Saving 16
Saving 17
Saving 18
Saving 19
Saving 20
Saving 24
Saving 25
Saving 26


In [183]:
# prepare all_the_bds to be put into a table -- organize by ra, dec later because mc_results depends on this order
obj_table = df.copy()

obj_table = obj_table.drop(['t0', 'pi', 'mu_alpha', 'mu_delta', 'pm_ra', 'pm_dec', 'pm_mu_alpha', 'pm_mu_delta', 'pm_pi'], axis=1)

In [184]:
# create duplicate array of all_events to only grab the lowest Mjup event
all_events_by_mjup = all_events.sort_values(by=['delta_m'], ascending=True)

# lists that will append
min_times, mags, seps, seps_e, delt_ms, delt_ms_e, times, times_e = list(), list(), list(), list(), list(), list(), list(), list()

for index, row in df.iterrows():
    row_id = row.id
    
    if row_id in mc_results.keys():
        # previous code used i, setting as i for ease
        i = row_id
        
        # calc data from MC
        delt_m, delt_m_e = np.mean(mc_results[i][0]), np.std(mc_results[i][0])
        sep, sep_e = np.mean(mc_results[i][1]), np.std(mc_results[i][1])
        time, time_e = np.mean(mc_results[i][2]), np.std(mc_results[i][2])

        # round calculated data
        sep, sep_e = round(sep, 2), round(sep_e, 2)
        delt_m, delt_m_e = round(delt_m, 2), round(delt_m_e, 2)
        time, time_e = round(time, 4), round(time_e, 4)


    # if not an object with an event, add "-"
    else:
        sep, sep_e, delt_m, delt_m_e, time, time_e = '-', '-', '-', '-', '-', '-'


    seps.append(sep)
    seps_e.append(sep_e)
    delt_ms.append(delt_m)
    delt_ms_e.append(delt_m_e)
    times.append(time)
    times_e.append(time_e)

# add to df
#obj_table['t_min'] = min_times

obj_table['Sep_min'] = seps
obj_table['Sep_min_e'] = seps_e
obj_table['Delta_m'] = delt_ms
obj_table['Delta_m_e'] = delt_ms_e

obj_table['t_min'] = times
obj_table['t_min_e'] = times_e


# now rearrange by ra, dec
obj_table = obj_table.sort_values('ra')

In [185]:
# reorder columns
cols = ['object_name', 'ra', 'dec', 'JmagL', 'e_JmagL', 'Gmag', 'e_Gmag', 't_min', 't_min_e', 'Sep_min', 'Sep_min_e', 'Delta_m', 'Delta_m_e']

obj_table = obj_table[cols]

# rename Mag
#obj_table = obj_table.rename(columns = {'Mag_BS': 'Mag_bs'})

In [212]:
# make table helpful for overleaf
obj_table_latex = obj_table.copy()

obj_table_latex = obj_table_latex[obj_table_latex.t_min_e != '-']

# combine into plus minuses
obj_table_latex.Sep_min = obj_table_latex.Sep_min.astype(str) + '$\pm$' + obj_table_latex.Sep_min_e.astype(str)
obj_table_latex.JmagL = obj_table_latex.JmagL.astype(str) + '$\pm$' + obj_table_latex.e_JmagL.astype(str)
obj_table_latex.Gmag = obj_table_latex.Gmag.astype(str) + '$\pm$' + obj_table_latex.e_Gmag.astype(str)
obj_table_latex.Delta_m = obj_table_latex.Delta_m.astype(str) + '$\pm$' + obj_table_latex.Delta_m_e.astype(str)
obj_table_latex.t_min = obj_table_latex.t_min.astype(str) + '$\pm$' + obj_table_latex.t_min_e.astype(str)

# drop error columns
obj_table_latex = obj_table_latex.drop(['e_JmagL', 'e_Gmag', 't_min_e', 'Sep_min_e', 'Delta_m_e', 'ra', 'dec'], axis=1)

# rename columns
rename = {'object_name': 'Object Name', 'JmagL': '$J_{\text{bd}}$', 'Gmag': '$G_{\text{bs}}$', 't_min': '$t_{\text{min}}$', 'Sep_min': '$\theta_{\text{min}}$', 'Delta_m': 'Pred. Mass Unc.'}

obj_table_latex = obj_table_latex.rename(columns = rename)

In [213]:
obj_table_latex

,Object Name,$J_{\text{bd}}$,$G_{\text{bs}}$,$t_{\text{min}}$,$\theta_{\text{min}}$,Pred. Mass Unc.
16,LSR_J0011+5908,9.95$\pm$0.02,20.131$\pm$0.0061,2032.717$\pm$0.0163,900.97$\pm$52.08,213.97$\pm$12.37
0,PSO_J076.7092+52.6087,15.75$\pm$0.07,18.694$\pm$0.0028,2044.7501$\pm$0.5043,99.12$\pm$83.51,43.01$\pm$36.23
1,WISEA_J053257.29+041842.5,15.44$\pm$0.06,17.4666$\pm$0.0014,2052.5565$\pm$0.1201,90.39$\pm$58.87,73.5$\pm$47.87
17,2MASS_J05441150-2433018,12.53$\pm$0.02,18.6746$\pm$0.0025,2021.4196$\pm$0.0286,192.91$\pm$17.81,102.75$\pm$9.49
2,2MASS_J05591914-1404488,13.8$\pm$0.02,20.0007$\pm$0.0081,2056.2616$\pm$0.1538,95.98$\pm$67.15,25.56$\pm$17.88
3,WISE_J070159.79+632129.2,15.79$\pm$0.07,20.5713$\pm$0.0094,2061.6376$\pm$0.6916,206.79$\pm$131.45,87.87$\pm$55.86
4,DENIS-P_J0751164-253043,13.16$\pm$0.02,20.9383$\pm$0.0162,2045.7003$\pm$0.1499,377.83$\pm$50.46,172.61$\pm$23.05
18,LHS_3003,9.97$\pm$0.03,20.8804$\pm$0.0144,2057.5324$\pm$0.2407,976.97$\pm$128.77,178.75$\pm$23.56
19,2MASS_J15485834-1636018,13.89$\pm$0.03,19.6715$\pm$0.0044,2050.461$\pm$0.3998,315.52$\pm$60.19,179.82$\pm$34.31
5,VVV_BD001,13.4$\pm$0.03,20.8126$\pm$0.019,2037.0765$\pm$0.1846,201.93$\pm$86.67,91.12$\pm$39.11
